# Impementazione dell'esercizio sulla test segmentation

## Step algoritmo:

- Dividere il file per righe, le righe guideranno l'esercizio in quanto ai tali ci sarà associata la riga di riferimento;
- Contare il valore di co-occorrenza per ogni frase:
  - Il valore di co-occorrenza è la somma dei valori di co-occorrenza per ogni parola in una frase;

### Tagli corretti (ita)
* 36/37 - taglio arg 1/2
* 56/57 - taglio arg 2/3

### Tagli corretti (en)

- 59/60
- 102/103

In [17]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from gensim.corpora.dictionary import Dictionary
from gensim import models
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random

In [2]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

def get_text_from_file(path):
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file
    

# Date due frasi ritorna il valore di cooccorrenza


Organizziamo le parole in una lista di liste, in qui ogni lista corrisponderà alle parole di una riga tokenizzate.

In [15]:
file = get_text_from_file('../res/segmentation_eng.txt')
c = Counter()
num_lines = sum(1 for line in open('../res/segmentation_eng.txt')) # Number of lines in the file

for row in file:
    c.update(row)
    
print(c.most_common(30))

[('quantum', 63), ('gorilla', 34), ('gorillas', 31), ('classical', 16), ('algorithm', 16), ('silverback', 15), ('females', 14), ('computers', 14), ('males', 13), ('computer', 13), ('algorithms', 13), ('male', 12), ('also', 12), ('problems', 12), ('western', 11), ('ft', 11), ('years', 10), ('mountain', 10), ('known', 10), ('computing', 10), ('eastern', 9), ('groups', 9), ('kg', 8), ('troops', 8), ('lowland', 8), ('adult', 8), ('may', 8), ('tend', 7), ('live', 7), ('model', 7)]


Calcolo il valore di cooccorrenza per ogni segmento

In [26]:
cut = random.randint(0, num_lines)
c1,c2 = Counter(), Counter()
i=1

for row in file:
    if(i < cut):
        c1.update(row)
    else:
        c2.update(row)
        
    i+=1

print("The cut is at line: ", cut)
print(c1.most_common(20), c2.most_common(20))

The cut is at line:  28
[('gorillas', 17), ('gorilla', 12), ('ft', 11), ('kg', 8), ('species', 6), ('western', 5), ('wild', 5), ('lb', 5), ('adult', 5), ('eastern', 4), ('living', 4), ('arm', 4), ('live', 4), ('lowland', 4), ('male', 4), ('forests', 3), ('africa', 3), ('humans', 3), ('metres', 3), ('called', 3)] [('quantum', 63), ('gorilla', 22), ('classical', 16), ('algorithm', 16), ('gorillas', 14), ('computers', 14), ('females', 13), ('computer', 13), ('algorithms', 13), ('also', 12), ('silverback', 12), ('problems', 12), ('males', 11), ('computing', 10), ('known', 9), ('groups', 9), ('mountain', 8), ('male', 8), ('years', 7), ('may', 7)]


In [1]:
with open('../res/segmentation.txt', 'r') as f:
    
    # Trasformo il file in una lista di liste di parole, la lista di parole corrisponde alla frase
    s = file2list(f)
    print(s)
    # length_p = len(s)
    # # Calcolo la cooccorrenza tra le frasi
    # cooc = cooccorence(s)
    # # print(cooc)
    # n_tagli = 3
    # tagli = pos_tagli(length_p, n_tagli)
    
    # val = [0, 0, 0]
    # for i in range(1, len(val)+1): # 3
    #     for j in range(tagli[i-1], tagli[i]): # in base a quante righe ci sono nel prima del taglio
    #         val[i] = val[i] + cooc[j][0]
            
    # print(val)

NameError: name 'file2list' is not defined